In [1]:
import os
import sys
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, roc_curve
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from src.preprocessing.preprocessing  import data_wrangling, ToDummy, CardinalityReducer, TeEncoder
# from src.modeling.supervided_models import LGBMModel, NNModel, LSTMNNModel, CATModel

In [5]:
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format #evita que muestre notacion cientifica
pd.set_option('display.max_columns', None)
np.set_printoptions(suppress=True) #evita mostrar notacion cientifica

In [6]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
mpl.rcParams['figure.figsize'] = (12, 10)

## Leer Archivos

In [7]:
print("Read data - Inicio")

Read data - Inicio


In [8]:
df = pd.read_parquet('tmp/df_tsfel_variables.parquet')

In [9]:
df.rename(columns={'id_usuario':'uc'}, inplace=True)
df.rename(columns={'id':'index'}, inplace=True)
df.rename(columns={'is_fraud':'target'}, inplace=True)

In [10]:
df.head()

,uc,12_anterior,11_anterior,10_anterior,9_anterior,8_anterior,7_anterior,6_anterior,5_anterior,4_anterior,3_anterior,2_anterior,1_anterior,date_fizcalizacion,cant_ttarifa,departamento,municipio,zona,tipo_tarifa,contrato,date,cod_mat,target,cant_null,eliminar,unidad_de_lectura,codigo_postal,fecha_de_alta,no_de_poste,tarfia,multiplicador,actividad_economica,kw_cont,medidor_interior,folio,nivel_de_tension,index,0_ECDF Percentile Count_0,0_ECDF Percentile Count_1,0_ECDF Percentile_0,0_ECDF Percentile_1,0_ECDF_0,0_ECDF_1,0_ECDF_2,0_ECDF_3,0_ECDF_4,0_ECDF_5,0_ECDF_6,0_ECDF_7,0_ECDF_8,0_ECDF_9,0_Histogram_0,0_Histogram_1,0_Histogram_2,0_Histogram_3,0_Histogram_4,0_Histogram_5,0_Histogram_6,0_Histogram_7,0_Histogram_8,0_Histogram_9,0_Interquartile range,0_Kurtosis,0_Max,0_Mean,0_Mean absolute deviation,0_Median,0_Median absolute deviation,0_Min,0_Root mean square,0_Skewness,0_Standard deviation,0_Variance,0_Absolute energy,0_Area under the curve,0_Autocorrelation,0_Centroid,0_Entropy,0_Mean absolute diff,0_Mean diff,0_Median absolute diff,0_Median diff,0_Negative turning points,0_Neighbourhood peaks,0_Peak to peak distance,0_Positive turning points,0_Signal distance,0_Slope,0_Sum absolute diff,0_Total energy,0_Zero crossing rate,0_FFT mean coefficient_0,0_FFT mean coefficient_1,0_FFT mean coefficient_2,0_FFT mean coefficient_3,0_FFT mean coefficient_4,0_FFT mean coefficient_5,0_FFT mean coefficient_6,0_Fundamental frequency,0_Human range energy,0_LPCC_0,0_LPCC_1,0_LPCC_10,0_LPCC_11,0_LPCC_12,0_LPCC_2,0_LPCC_3,0_LPCC_4,0_LPCC_5,0_LPCC_6,0_LPCC_7,0_LPCC_8,0_LPCC_9,0_MFCC_0,0_MFCC_1,0_MFCC_10,0_MFCC_11,0_MFCC_2,0_MFCC_3,0_MFCC_4,0_MFCC_5,0_MFCC_6,0_MFCC_7,0_MFCC_8,0_MFCC_9,0_Max power spectrum,0_Maximum frequency,0_Median frequency,0_Power bandwidth,0_Spectral centroid,0_Spectral decrease,0_Spectral distance,0_Spectral entropy,0_Spectral kurtosis,0_Spectral positive turning points,0_Spectral roll-off,0_Spectral roll-on,0_Spectral skewness,0_Spectral slope,0_Spectral spread,0_Spectral variation,0_Wavelet absolute mean_0,0_Wavelet absolute mean_1,0_Wavelet absolute mean_2,0_Wavelet absolute mean_3,0_Wavelet absolute mean_4,0_Wavelet absolute mean_5,0_Wavelet absolute mean_6,0_Wavelet absolute mean_7,0_Wavelet absolute mean_8,0_Wavelet energy_0,0_Wavelet energy_1,0_Wavelet energy_2,0_Wavelet energy_3,0_Wavelet energy_4,0_Wavelet energy_5,0_Wavelet energy_6,0_Wavelet energy_7,0_Wavelet energy_8,0_Wavelet entropy,0_Wavelet standard deviation_0,0_Wavelet standard deviation_1,0_Wavelet standard deviation_2,0_Wavelet standard deviation_3,0_Wavelet standard deviation_4,0_Wavelet standard deviation_5,0_Wavelet standard deviation_6,0_Wavelet standard deviation_7,0_Wavelet standard deviation_8,0_Wavelet variance_0,0_Wavelet variance_1,0_Wavelet variance_2,0_Wavelet variance_3,0_Wavelet variance_4,0_Wavelet variance_5,0_Wavelet variance_6,0_Wavelet variance_7,0_Wavelet variance_8,mean_3,cant_ceros_3,max_cant_ceros_seg_3,slope_3,min_cons3,max_cons3,std_cons3,var_cons3,skew_cons3,mean_6,cant_ceros_6,max_cant_ceros_seg_6,slope_6,min_cons6,max_cons6,std_cons6,var_cons6,skew_cons6,kurt_cons6,mean_12,cant_ceros_12,max_cant_ceros_seg_12,slope_12,min_cons12,max_cons12,std_cons12,var_cons12,skew_cons12,kurt_cons12
0,100035,153.00,125.00,117.00,120.00,128.00,80.00,105.00,123.00,101.00,111.00,99.00,96.00,2022-01-01,1,guatemala,villa_nueva,zona_3,TS,100035,2022-01-01,41-0069,0,0,0,708-008,1064,2001-07-11,238366,R11,1.00,83_009,0.00,sin_dato,708-10329-000,2.00,0,2.00,9.00,96.00,123.00,0.08,0.17,0.25,0.33,0.42,0.50,0.58,0.67,0.75,0.83,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23.00,0.09,153.00,113.17,14.50,114.00,12.00,80.00,114.60,0.32,18.07,326.64,157600.00,12.33,157600.00,0.05,1.00,16.82,-5.18,12.00,-3.00,3.00,0.00,73.00,3.00,185.64,-3.45,185.00,1432727.27,0.00,1.46,3.42,5.62,2.69,8.45,3.70,0.05,10.00,0.00,0.04,0.66,0.20,0.44,0.66,0.44,0.20,0.31,0.17,0.36,0.36,0.17,0.31,-11.58,-6.22,61.83,3.84,19.34,24.33,-139.50,124.05,75.00,-34.39,76.72,-84.49,0.03,40

In [11]:
df.shape

(30932, 207)

In [12]:
print("Read data - Fin")

Read data - Fin


## Data Wrangling

In [13]:
%%time
df = data_wrangling(df,12)

CPU times: user 289 ms, sys: 49.3 ms, total: 338 ms
Wall time: 337 ms


In [14]:
df.cod_mat.isnull().sum()

0

## Preprocesar variables

In [33]:
cols_uc = ['departamento','codigo_postal','actividad_economica','tipo_tarifa','tarfia','nivel_de_tension','cod_mat','cant_ttarifa','medidor_interior']

### Dummy variables

In [15]:
print(f"Cardinalidad de departamento : {df.departamento.nunique()}")

Cardinalidad de departamento : 5


In [17]:
vars_dummy = ['departamento']
obj_todummy = ToDummy(vars_dummy)
df_dummy = obj_todummy.fit_transform(df[vars_dummy])

In [18]:
df_dummy.head()

,dummy_departamento_chimaltenango,dummy_departamento_escuintla,dummy_departamento_guatemala,dummy_departamento_sacatepequez,dummy_departamento_santa_rosa
0,0,0,1,0,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,0,1,0,0


### Ordinal encoder

In [19]:
print(f"Cardinalidad de codigo_postal : {df.codigo_postal.nunique()}")

Cardinalidad de codigo_postal : 70


In [20]:
vars_oe = ['codigo_postal']
obj_oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df_oe = obj_oe.fit_transform(df[vars_oe])

In [21]:
pd.DataFrame(df_oe, columns=['codigo_postal']).head()

,codigo_postal
0,35.00
1,3.00
2,9.00
3,28.00
4,24.00


### Reduccion de la cardinalidad

In [22]:
print(f"Cardinalidad de actividad_economica : {df.actividad_economica.nunique()}")

Cardinalidad de actividad_economica : 174


In [23]:
vars_cr = ['actividad_economica']
obj_cr = CardinalityReducer(threshold=0.001)
df_cr = obj_cr.fit_transform(df[vars_cr])

In [24]:
df_cr.actividad_economica.nunique()

34

In [26]:
df_cr.actividad_economica.unique()

array(['83_009', 'otros', '83_002', '87_006', 'sin_dato', '83_005',
       '83_006', '87_009', '74_002', '83_004', '83_007', '87_003',
       '59_003', '83_003', '84_003', '83_001', '87_004', '86_021',
       '59_001', '73_005', '59_005', '73_002', '83_008', '59_004',
       '49_001', '59_002', '62_008', '87_001', '88_009', '55_002',
       '74_001', '49_004', '88_008', '85_006'], dtype=object)

### [Target-encoding](https://towardsdatascience.com/dealing-with-categorical-variables-by-using-target-encoder-a0f1733a4c69)

### !!! Tener cuidad con el !!!

Tratar de igual manera que un normalizador!!

In [27]:
print(f"Cardinalidad de cod_mat : {df.cod_mat.nunique()}")

Cardinalidad de cod_mat : 21


In [28]:
vars_te = ['cod_mat']
obj_te = TeEncoder(vars_te,w=5)
df_te = obj_te.fit_transform(df[vars_te],df.target)

In [29]:
df_te.head()

,cod_mat
0,0.00
1,0.01
2,0.00
3,0.00
4,0.01


## [Combinacion de procesamientos](https://towardsdatascience.com/pipeline-columntransformer-and-featureunion-explained-f5491f815f)

- Pipeline
- ColumnTransformer


In [30]:
# Actividad economica
pipe_ae = Pipeline([
            ('cardinality_reducer', CardinalityReducer(threshold=0.001)),
            ('te',ToDummy(['actividad_economica']))
        ])

# Actividad tarfia
pipe_tarifa = Pipeline([
            ('cardinality_reducer', CardinalityReducer(threshold=0.001)),
            ('te',TeEncoder(['tarfia'],w=50))
        ])

In [34]:
vars_dummy = ['departamento','tipo_tarifa','medidor_interior']
vars_enc = ['codigo_postal','nivel_de_tension']
t_features = [
    ('dummy_var', ToDummy(vars_dummy), vars_dummy),
    ('enc_var', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), vars_enc),
    ('te_cod_mat', TeEncoder(['cod_mat'],w=10), ['cod_mat']),
    ('te_ae', pipe_ae, ['actividad_economica']),
    ('te_tarifa', pipe_tarifa, ['tarfia']),
    ]

preprocessor = ColumnTransformer(transformers= t_features,remainder='passthrough')

In [35]:
df_preprocesor = preprocessor.fit_transform(df[cols_uc],df.target)

In [36]:
pd.DataFrame(df_preprocesor)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,35.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,1.00
1,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,3.00,3.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
2,0.00,0.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,9.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
3,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,28.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
4,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,24.00,3.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30927,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,27.00,3.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
30928,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,27.00,3.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
30929,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,27.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
30930,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,11.00,3.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.01,1.00
